In [1]:
import numpy as np
import time
import math
import cv2

In [2]:
labelsPath = "./coco.names"
LABELS = open(labelsPath).read().strip().split("\n")
LABELS

['person',
 'bicycle',
 'car',
 'motorcycle',
 'airplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'couch',
 'potted plant',
 'bed',
 'dining table',
 'toilet',
 'tv',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush']

In [3]:
np.random.seed(42)
COLORS = np.random.randint(0, 255, size = (len(LABELS), 3), dtype = 'uint8')
COLORS

array([[102, 220, 225],
       [ 95, 179,  61],
       [234, 203,  92],
       [  3,  98, 243],
       [ 14, 149, 245],
       [ 46, 106, 244],
       [ 99, 187,  71],
       [212, 153, 199],
       [188, 174,  65],
       [153,  20,  44],
       [203, 152, 102],
       [214, 240,  39],
       [121,  24,  34],
       [114, 210,  65],
       [239,  39, 214],
       [244, 151,  25],
       [ 74, 145, 222],
       [ 14, 202,  85],
       [145, 117,  87],
       [184, 189, 221],
       [116, 237, 109],
       [ 85,  99, 172],
       [226, 153, 103],
       [235, 146,  36],
       [151,  62,  68],
       [181, 130, 160],
       [160, 166, 149],
       [  6,  69,   5],
       [ 52, 253, 112],
       [ 14,   1,   3],
       [ 76, 248,  87],
       [233, 212, 184],
       [235, 245,  26],
       [213, 157, 253],
       [ 68, 240,  37],
       [219,  91,  54],
       [129,   9,  51],
       [  0, 191,  20],
       [140,  46, 187],
       [147,   1, 254],
       [ 20, 153, 243],
       [ 46, 160

In [4]:
weightsPath = "./yolov3.weights"
configPath = "./yolov3.cfg"

net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
net

<dnn_Net 000002583A265630>

In [7]:
cap = cv2.VideoCapture(0)

while(cap.isOpened()):
    ret, image = cap.read()
    (H, W) = image.shape[:2]
    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416,416), swapRB = True, crop = False)
    net.setInput(blob)
    start = time.time()
    layerOutputs = net.forward(ln)   
    end = time.time()
    print("Frame Prediction Time : {:.6f} seconds".format(end - start))
    boxes = []
    confidences = []
    classIDs = []
    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > 0.1 and classID == 0:
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)
                
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5,0.3)
    ind = []
    for i in range(0, len(classIDs)):
        if(classIDs[i] == 0):
            ind.append(i)
            
    a = []
    b = []

    if len(idxs) > 0:
        for i in idxs.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            a.append(x)
            b.append(y)
    
    distance = []
    nsd = []

    for i in range(0, len(a) - 1):
        for k in range(1, len(a)):
            if(k == i):
                break
            else:
                x_dist = (a[k] - a[i])
                y_dist = (b[k] - b[i])
            
                d = math.sqrt(x_dist * x_dist + y_dist * y_dist)
                distance.append(d)
            
                if(d <= 100):
                    nsd.append(i)
                    nsd.append(k)
                    nsd = list(dict.fromkeys(nsd))
                    print(nsd)
                    
    color = (0, 0, 255)
    for i in nsd:
        (x, y) = (boxes[i][0], boxes[i][1])
        (w, h) = (boxes[i][2], boxes[i][3])
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
        text = "Alert"
        cv2.putText(image, text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
    color = (0, 255, 0)
    if len(idxs) > 0:
        for i in idxs.flatten():
            if (i in nsd):
                break
            else:
                (x, y) = (boxes[i][0], boxes[i][1])
                (w, h) = (boxes[i][2], boxes[i][3])
                cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
                text = 'OK'
                cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,0.5, color, 2)
                
    cv2.imshow("Social Distancing Detector", image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        

cap.release()
cv2.destroyAllWindowa()
    
        

Frame Prediction Time : 0.886630 seconds
Frame Prediction Time : 0.899594 seconds
Frame Prediction Time : 0.818812 seconds
Frame Prediction Time : 0.822796 seconds
Frame Prediction Time : 0.854713 seconds
Frame Prediction Time : 0.864688 seconds
Frame Prediction Time : 0.883625 seconds
Frame Prediction Time : 0.881636 seconds
Frame Prediction Time : 0.851701 seconds
Frame Prediction Time : 0.832762 seconds
Frame Prediction Time : 0.855715 seconds
Frame Prediction Time : 0.899595 seconds
Frame Prediction Time : 1.048191 seconds
Frame Prediction Time : 0.952444 seconds
Frame Prediction Time : 0.959435 seconds
Frame Prediction Time : 0.837758 seconds
Frame Prediction Time : 0.837758 seconds
Frame Prediction Time : 0.860699 seconds
Frame Prediction Time : 0.852716 seconds
Frame Prediction Time : 0.795872 seconds
[0, 1]
Frame Prediction Time : 2.200116 seconds
Frame Prediction Time : 0.824785 seconds
Frame Prediction Time : 1.783228 seconds
Frame Prediction Time : 1.610696 seconds
Frame Pre

KeyboardInterrupt: 